In [142]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [143]:
df = pd.read_csv('../../Data Files/Raw Data/users_ratings.csv')
df.head(5)

,UserID,Title,Year,Description,Directors,Stars,Viewer_Advisory,Duration,Genre,Votes,Movie_Rating,User_Rating
0,ur117926588,A Nightmare on Elm Street 2: Freddy's Revenge,1985,A teenage boy is haunted in his dreams by dece...,['Jack Sholder'],"['Robert Englund', 'Mark Patton', 'Kim Myers',...",R,1 hr 27 min,Horror,72454.0,5.4,5
1,ur117926588,Elvis,2022,The life of American music icon,['Baz Luhrmann'],"['Tom Hanks', 'Austin Butler', 'Olivia DeJonge...",PG-13,2 hr 39 min,"Biography, Drama, Music",196677.0,7.3,8
2,ur117926588,Where Eagles Dare,1968,Allied agents stage a daring raid on a castle ...,['Brian G. Hutton'],"['Richard Burton', 'Clint Eastwood', 'Mary Ure...",M,2 hr 38 min,"Action, Adventure, War",59856.0,7.6,9
3,ur117926588,A Nightmare on Elm Street,1984,Teenager Nancy Thompson must uncover the dark ...,['Wes Craven'],"['Heather Langenkamp', 'Johnny Depp', 'Robert ...",R,1 hr 31 min,Horror,245805.0,7.4,9
4,ur117926588,Eagle Eye,2008,Jerry and Rachel are two strangers thrown toge...,['D.J. Caruso'],"['Shia LaBeouf', 'Michelle Monaghan', 'Rosario...",PG-13,1 hr 58 min,"Action, Mystery, Thriller",189895.0,6.6,5


In [144]:
df_rating = df[['UserID', 'Title', 'User_Rating']]
df_rating

,UserID,Title,User_Rating
0,ur117926588,A Nightmare on Elm Street 2: Freddy's Revenge,5
1,ur117926588,Elvis,8
2,ur117926588,Where Eagles Dare,9
3,ur117926588,A Nightmare on Elm Street,9
4,ur117926588,Eagle Eye,5
...,...,...,...
15590,ur4532636,Jackie Brown,10
15591,ur4532636,Beauty and the Beast,6
15592,ur4532636,Dirty Pretty Things,9
15593,ur4532636,"Easy Riders, Raging Bulls: How the Sex, Drugs ...",7


In [145]:
ratings_count = df_rating.groupby(by="Title")['User_Rating'].count().reset_index().rename(columns={'User_Rating':'num_rating'})[['Title','num_rating']]

In [146]:
ratings_count.sample(5)

,Title,num_rating
10086,Weary River,1
602,American Dad! - Faking Bad,1
29,12 Monkeys - Yesterday,1
7301,The Best Years of Our Lives,2
9859,Two Doors Down - Alan and Michelle,1


In [147]:
ratings_count['num_rating'].describe()

count    10347.000000
mean         1.507200
std          0.966489
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         10.000000
Name: num_rating, dtype: float64

In [148]:
ratings_count['num_rating'].quantile(np.arange(.8,1,0.01))

0.80    2.0
0.81    2.0
0.82    2.0
0.83    2.0
0.84    2.0
0.85    2.0
0.86    2.0
0.87    2.0
0.88    3.0
0.89    3.0
0.90    3.0
0.91    3.0
0.92    3.0
0.93    3.0
0.94    3.0
0.95    4.0
0.96    4.0
0.97    4.0
0.98    5.0
0.99    5.0
Name: num_rating, dtype: float64

In [149]:
votes_count_threshold = 2
ratings_total = pd.merge(df_rating,ratings_count,on='Title',how='left')
ratings_top = ratings_total.query('num_rating > @votes_count_threshold')

In [150]:
ratings_top.drop_duplicates(['UserID', 'Title'], inplace=True)
#ratings_total[ratings_total.duplicated(subset=['UserID', 'Title'], keep='first')][:30]

C:\Users\alici\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [151]:
df_for_knn = ratings_top.pivot(index='Title',columns='UserID',values='User_Rating').fillna(0)
df_for_knn

UserID,ur117926588,ur15298231,ur17646017,ur1994077,ur22171966,ur4532636
Title,,,,,,
'71,9.0,6.0,0.0,0.0,0.0,8.0
10,7.0,0.0,0.0,6.0,0.0,4.0
10 Cloverfield Lane,9.0,0.0,0.0,5.0,8.0,6.0
"10,000 BC",4.0,0.0,10.0,0.0,0.0,2.0
12 Angry Men,8.0,0.0,9.0,7.0,0.0,10.0
...,...,...,...,...,...,...
Zero Dark Thirty,0.0,0.0,9.0,0.0,6.0,9.0
Zodiac,0.0,6.0,8.0,7.0,9.0,9.0
Zombieland,0.0,0.0,9.0,7.0,8.0,9.0


In [152]:
df_for_knn.shape

(1320, 6)

In [153]:
df_for_knn_sparse = csr_matrix(df_for_knn.values)

In [154]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(df_for_knn_sparse)
query_index = np.random.choice(df_for_knn.shape[0])
distances, indices = model_knn.kneighbors(df_for_knn.iloc[query_index,:].values.reshape(1,-1),n_neighbors=10)
for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendations for movie: {0}\n".format(df_for_knn.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}".format(i,df_for_knn.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for movie: Green Room

1: Tron, with distance of 0.0025538748798825672
2: Robot Jox, with distance of 0.0029090970728533616
3: The Running Man, with distance of 0.005630826364807273
4: The Rocketeer, with distance of 0.006046780130839369
5: Bone Tomahawk, with distance of 0.0065353705359951775
6: Army of Darkness, with distance of 0.007456544723359437
7: Chopping Mall, with distance of 0.0077129600060512615
8: Trading Places, with distance of 0.015047303488903818
9: John Wick, with distance of 0.015471085293004894


https://www.kaggle.com/code/sirpunch/build-knn-and-apriori-based-recommendation-engines